In [ ]:
 running = 0    # 0: var, 1: and/or, 2: not
for t in instr[2:]:
    if (t in declared_vars) or (t in declared_ids):
        running = 0
    elif t == ')':
        if running > 0:
            raise Exception(f"Invalid assignment {instr}")
    elif t == '(':
        if running == 0:
            raise Exception(f"Invalid assignment {instr}")
    elif t in ('and', 'or'):
        if running != 0:
            raise Exception(f"Invalid assignment {instr}")
        running = 1
    elif t == 'not':
        running = 2
    else:
        raise Exception(f"Invalid token in assignment: {t}")

In [ ]:
def check_validity_recursively(expr, declared_vars, declared_ids):
    
    def check_validity(expr, declared_vars, declared_ids):

        node_type = None
        i = 0
        running = -1   # 0: var, 1: and/or, 2: not
        while i < len(expr):
            token = expr[i]
            
            # OPEN parens --> look for matching closing parens
            if token == '(':
                if running == 0:
                    raise Exception(f'Invalid assignment {expr}')
                open_parens = 1
                for j in range(i + 1, len(expr)):
                    if expr[j] == '(':
                        open_parens += 1
                    elif expr[j] == ')':
                        open_parens -= 1
                    if open_parens == 0:
                        # Solve the sub-expr within the parentheses
                        if expr[i + 1:j] == []:
                            raise Exception(f'Empty parentheses at position {i} in <{expr}>')
                        
                        check_validity(expr[i + 1:j])
                        i = j  # Move the index to after the closing ')'
                        break
            
            # CLOSING parens
            elif token == ')':
                raise Exception(f'Unmatched closing parenthesis at position {i} in <{expr}>')
            
            # IDENTIFIER / VARIABLE
            elif (t in declared_vars) or (t in declared_ids):
                if running == 0:
                    raise Exception(f'Invalid syntax at position {i} in <{expr}>')
                running = 0
            
            # NOT
            elif token == 'not':
                if running != -1:
                    raise Exception(f'Invalid syntax at position {i} in <{expr}>')
                running = 2
            
            # AND / OR
            elif token in ('and', 'or'):
                if running != 0:
                    raise Exception(f'Invalid syntax at position {i} in <{expr}>')
            
            
            else:
                raise Exception(f'Invalid token {token} at position {i} in <{expr}>')
                
            i += 1
        if running > 0:
            raise Exception(f'Invalid assignment {expr}')
        
        return 
    
    return check_validity(expr)

In [57]:
f = 'j2 j1 0e 0 1 0'

re.sub(' ', '  ', f)
re.sub(r'\n[01]\n', '  ', f)

'j2 j1 0e 0 1 0'

In [10]:
import os
import re

path1 = r"C:\Users\giaco\Desktop\240911_20875-Software-Engineering\HW01\outputs"
path2 = r"C:\Users\giaco\Downloads\outputs"
for file in os.listdir(path1):
    print(file)
    # JACK  
    f1 = open(path1 + "/" + file, 'r').read()
    f1 = f1.replace(' ', '').replace('\n\n', '\n')
    f1 = f1.splitlines()[1:]
    f1 = '\n'.join(f1)
    f1 = re.sub(r'[^01]', '', f1)

    # VEZZO
    f2 = open(path2 + '/' + file[:-4] + '_output.txt', 'r', encoding='utf-16').read()
    f2 = re.sub(r'---.+---', '', f2)
    f2 = f2.splitlines()[1:]
    f2 = '\n'.join(f2)
    f2 = f2.replace(' ', '')
    f2 = re.sub(r'[^01]', '', f2)

    for i in range(len(f2)):
        if f1[i] != f2[i]:
            print(f1[i], f2[i])
            print(i)


ag24_00.txt
ag24_01.txt
ag24_02.txt
ag24_03.txt
ag24_04.txt
ag24_05.txt
ag24_06.txt
ag24_07.txt
ag24_08.txt
ag24_09.txt
ag24_10.txt
ag24_11.txt
ag24_12.txt
ag24_13.txt
ag24_14.txt
ag24_15.txt


In [ ]:
expr = ['x', 'and', '(', 'not', '(', 'x', 'or', '(', 'y', 'and', '(', 'not', 'z', ')', ')', ')', 'or', 'w', ')']

vars = ['x', 'y', 'z', 'w']

class Node:
    def __init__(self, value):
        self.v = value      # variable name
        self.low= None      # var = 0
        self.high = None    # var = 1

def build_BDD(expr, vars):

    def build_BDD_rec(expr, vars, table):
        current_var = vars.pop()
        i = 0
        while i < len(expr):
            if expr[i] == current_var:
                
            
